In [42]:
import logging
import openai
import praw
import os
from dotenv import load_dotenv
import tweepy
import json
import time
import requests
from datetime import datetime 
load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [43]:
REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')
REDDIT_USERNAME = os.getenv('REDDIT_USERNAME')
REDDIT_PASSWORD = os.getenv('REDDIT_PASSWORD')

# OpenAI API key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
# client = openai.Client()

# Twitter API credentials
TWITTER_CONSUMER_KEY = os.getenv('TWITTER_CONSUMER_KEY')
TWITTER_CONSUMER_SECRET = os.getenv('TWITTER_CONSUMER_SECRET')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_SECRET = os.getenv('TWITTER_ACCESS_SECRET')

# Social Searcher API key
SOCIAL_SEARCHER_API_KEY = os.getenv('SOCIAL_SEARCHER_API_KEY')

# Create Reddit API client (PRAW) - way to interact with Reddit API
reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID,
                     client_secret=REDDIT_CLIENT_SECRET,
                     user_agent=REDDIT_USER_AGENT,
                     username=REDDIT_USERNAME,
                     password=REDDIT_PASSWORD,
                     ratelimit_seconds=5)
    
# Create Twitter API client
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_SECRET)
twitter_api = tweepy.API(auth)



In [34]:
business_name = "Amplifire AI"
business_description = "Generate more leads from social media across LinkedIn, Facebook, X/Twitter, Reddit, and Instagram! We automate finding customers, and our AI replies to frustrated users' posts with genuinely valuable replies to help them. Then, we subtly provide your business with a solution to their problems in a way that doesn't come across as spammy!"
business_website = "https://amplifire-ai.vercel.app/"
business_location = ""
business_industry = "Artificial Intelligence"
business_tap = "Startups and small businesses seeking AI-powered"
business_keywords = "AI marketing automation, lead generation, AI business growth"


In [35]:
def generate_reddit_reply(post, business_name, business_description, website_link):
    description = post.selftext[:5000]
    response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": f"You are a thoughtful and insightful Reddit user who provides genuine and valuable responses to posts. Your goal is to assist users by offering advice, insights, or solutions. And naturally mention our business, {business_name}, in a way that adds value to the conversation. Include our link, {website_link}, if it fits naturally, explaining briefly how it might help with their issue. Avoid being overly promotional or spammy. Maintain a neutral and helpful tone throughout your response."
                },
                
                {
                    "role": "user",
                    "content": f"""
                        Please analyze the following Reddit post's title and description and craft a genuine and helpful reply that provides value to the user. 
                        Find a way to naturally recommend our business, if it is relevant and can be seamlessly integrated into your response. 
                        If you can't mention the business naturally, focus solely on providing valuable advice. Focus on speaking in a neutral tone and avoid using exclaimation marks. 
                        Please try to find a way to RECOMMEND our, in the reply naturally and preferably mention the, as well.
                        when promiting the business ensure you have mentioned the name. if the website link is relevent then add that too.
                        Everything after this point is information for you to use in your response. reply only with the generated comment and nothing else. 
                        make the english simple and smooth and in style of a comment
                        Title: {post.title}
                        Description: {description}
                        Our Business Name: {business_name}
                        Our Business Description: {business_description}
                        Our Website Link: {website_link}
                    """
                }
            ],
            max_tokens=300,
            temperature=0.7
        )
    return response.choices[0].message.content.strip()

def check_rate_limit(reddit):
    limits = reddit.auth.limits
    remaining = limits.get('remaining', None)
    reset_timestamp = limits.get('reset_timestamp', None)
    
    if remaining is not None and remaining <= 1:
        logging.warning(f"Rate limit reached. Waiting until reset.")
        reset_in = reset_timestamp - time.time()
        if reset_in > 0:
            logging.info(f"Waiting for {reset_in:.2f} seconds until rate limit resets.")
            time.sleep(reset_in)  # Wait until the rate limit resets
    else:
       print(f"Remaining API requests: {remaining}")
    

In [45]:

def fetch_reddit_leads(keyword, location, business_name, business_description, website_link):
    logging.info(f"Searching for keyword: {keyword}")
    # TODO: if already posted on specific post, don't post again. How can we keep track of this? For example if a user uses it one day and then again the next day, we don't want to post on the same post again.
    if location:
        logging.info(f"Location provided: {location}")
        keyword = f"{keyword} {location}"

    subreddit = reddit.subreddit("all")
    posts = {}
    
    # Load previously posted post IDs from a file to avoid duplicate posts
    try:
        with open('all_posts.json', 'r') as f:
            all_posts = json.load(f)
    except Exception as e:
        all_posts = {}

    # Set a limit for the number of new posts to comment on
    max_new_posts = 5
    new_posts_found = 0


    # Search through posts, but only comment on new ones
    for post in subreddit.search(keyword, limit=5):  # Increase the limit to search more posts
        logging.info(f"Processing post: {post.title}")

        # Skip if we've already posted in this exact post
        if post.id in all_posts:
            logging.info(f"Already processed this post: {post.title}, skipping...")
            continue  # Skip to the next post so we don't waste AI credits

        if post.archived:
            logging.info(f"Post is archived: {post.title}, skipping...")
            continue

        # Create the comment  
            
        comment = generate_reddit_reply(post, business_name, business_description, website_link)

        # Add the post ID to the list of all posts (before the try block)
        logging.info(f"Comment generated: {comment}") # Log the comment that was generated

        try:
            # Post the comment
            all_posts[post.id] = {
                'id': post.id,
                'description': post.title,
                'url': post.url,
                'timestamp': datetime.now().isoformat(),
                'comment': comment,
                # "comment_url": f"https://www.reddit.com{posted_comment.permalink}"
            }
            posted_comment = post.reply(comment)
            logging.info(f"Comment posted successfully on post: {post.title}")
            logging.info(f"Comment ID: {posted_comment.id}")
            logging.info(f"Comment URL: https://www.reddit.com{posted_comment.permalink}")

            # Increment the count of new posts found
            new_posts_found += 1

            # If we've found enough new posts, stop searching
            if new_posts_found >= max_new_posts:
                logging.info(f"Found {new_posts_found} new posts, stopping search.")
                break
            all_posts[post.id] = {
                'id': post.id,
                'description': post.title,
                'url': post.url,
                'timestamp': datetime.now().isoformat(),
                'comment': comment,
                # "comment_url": f"https://www.reddit.com{posted_comment.permalink}"
            }
        except Exception as e:
            logging.error(f"Error posting comment on post: {post.title}")
            logging.error(f"Error message: {str(e)}")

        

        # Save the updated list of posted post IDs to a file TODO: Improve this to not write the posts that haven't been replied too because of rate limits... We might just have to do not automatic posts and just show the user the posts and let them decide which ones to post on.
    with open('all_posts.json', 'w') as f: # by opening a file we overwrite the previous content
        json.dump(all_posts, f, indent=4) # add the list of posted posts to the file

        # Add a random delay between 30 and 60 seconds to avoid spam detection
        # delay = random.randint(5, 10)
        # logging.info(f"Waiting for {delay} seconds before posting the next comment...")
        # time.sleep(delay)
    
    return posts

In [46]:
fetch_reddit_leads(business_keywords, business_location, business_name, business_description, business_website)

ERROR:root:Error posting comment on post: How To Use AI In My Coaching Business
ERROR:root:Error message: RATELIMIT: "Looks like you've been doing that a lot. Take a break for 6 minutes before trying again." on field 'ratelimit'
ERROR:root:Error posting comment on post: The Future of Generative AI in Manufacturing: AI Innovations, Real-Life Examples and Challenges
ERROR:root:Error message: RATELIMIT: "Looks like you've been doing that a lot. Take a break for 6 minutes before trying again." on field 'ratelimit'
ERROR:root:Error posting comment on post: AI-Amplified Click Lead Generation using GO - AI: The Key to Exponential Growth
ERROR:root:Error message: RATELIMIT: "Looks like you've been doing that a lot. Take a break for 6 minutes before trying again." on field 'ratelimit'
ERROR:root:Error posting comment on post: MailerMonster – Review of the World’s First Gemini Ultra Autoresponder: Google’s Cutting-Edge AI for Generating Endless Content and Emails to Unlimited Subscribers, All Fr

{}

In [44]:
subreddit = reddit.subreddit("all")
for post in subreddit.search(business_keywords, limit=10):
    print(post.id)
    reply = post.reply("test")
    print("url: ",f"https://www.reddit.com{reply.permalink}")

1edkg15


RedditAPIException: RATELIMIT: "Looks like you've been doing that a lot. Take a break for 7 minutes before trying again." on field 'ratelimit'

In [25]:
check_rate_limit(reddit)

Remaining API requests: 994.0
